# Detect A-E Mismatch

## Train model to predict answer using Question + Explanation

Approach:
    - Question --> gpt2 --> Q_embeddings
        Explanations --> gpt2 --> E_embeddings
    - Merge Q+E --> Send to Classifier

In [1]:
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel
import pandas as pd
import math
import torch
import numpy as np

# https://github.com/Meelfy/pytorch_pretrained_BERT#usage

model = OpenAIGPTModel.from_pretrained('openai-gpt')
model.eval()
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


['hello</w>']

In [2]:
data = pd.read_csv('Post_processing.csv', index_col=0)

In [99]:
#%%time
import os


SAVEPATH = r'text_vectors\\'
def get_encoding(row,idx, col = None, encoded = True):
    if col == None:
        col = ['exp', 'qstr']
    for j in col:
        sent = row[j]
        sent_tok = tokenizer.tokenize(sent)
        sent_tok_idx = tokenizer.convert_tokens_to_ids(sent_tok)
        tok_tensor = torch.tensor([sent_tok_idx])
        if encoded == True:
            encoded = model(tok_tensor)
            encoded = encoded.detach().numpy()
        else:
            encoded = sent_tok_idx
        np.save(f'{SAVEPATH}{j}\\{j}_{str(idx)}', encoded)
    return 'Done'

#data['exp_enc'] = data['exp']
#data['qstr_enc'] = data['qstr']


for idx,row in data.iterrows():
    get_encoding(row,idx,['ans','pred'],False)
# data['qstr_enc'].apply((lambda x: get_encoding(x)))



data

ans  \
363036001              cake   
426070000             couch   
75560012                yes   
253810003               yes   
206878001               yes   
93236000                 no   
278601006               yes   
126995004            tennis   
192168002                no   
197022001                no   
62865003                yes   
243600002             chair   
302200002                no   
4296004             kitchen   
266124002             horse   
74603000                 no   
212559000               hay   
423116003               yes   
260150015          pancakes   
247484002       living room   
35525002                yes   
118432009                no   
519316004           bedroom   
449872015               yes   
366421000                no   
147921013                no   
69863000                 no   
272673007          bathroom   
279437000           bedroom   
554618006               yes   
...                     ...   
428231000            modern   
528866001               yes   
188386007               yes   
425077002               yes   
410627002               yes   
398385000                no   
544607001           frisbee   
171717004          bathroom   
154520073       flying kite   
291404001            winter   
204188003               yes   
309402000               yes   
300124001                no   
94168002                 no   
429290010  throwing frisbee   
420110010                no   
243912000               dog   
493905003            tennis   
412798004    playing tennis   
181799001               yes   
434673001            tennis   
503663011          bathroom   
286322004                no   
460927012               yes   
386707000                no   
302531003               yes   
102741005               yes   
507135001      fire hydrant   
209431000               yes   
568893000                no   

                                                         exp             pred  \
363036001  there is a pastry base which has been adorned ...             cake   
426070000                                        it is a bed            couch   
75560012                         there are clouds in the sky              yes   
253810003                                       it is yellow               no   
206878001                    it is a big portion of a plated              yes   
93236000                 the train is sitting at the station               no   
278601006                                       it is parked               no   
126995004                 the man is holding a tennis racket           tennis   
192168002  the man is wearing a suit and a woman to weari...               no   
197022001  there is a slice of yellow substance with hole...               no   
62865003                     it is dusty and looks like dirt              yes   
243600002                it is a flat and with a narrow seat            chair   
302200002                                it is a city street               no   
4296004                  there is a stove and a refrigerator          kitchen   
266124002                        it has a long mane and tail            horse   
74603000                     there are no people in the room               no   
212559000           they are eating plants with a wine place            grass   
423116003                  the kitchen is full of appliances               no   
260150015         it is cut in a slices and full of toppings        breakfast   
247484002                             there is a entry table      living room   
35525002               there is a kitchen and a coffee table              yes   
118432009            the sky is clear and the people are dry               no   
519316004                               there is a bed in it          bedroom   
449872015                          the sky is clear and blue              yes   
366421000                      the window is angled outwards               n

In [66]:
get_encoding(data.loc[291404001].exp).detach().numpy().size

5


4608

In [96]:
data = pd.read_pickle('Post_Processing_with_Encoding.pkl')
t = data['ans'].sample().item()
r = tokenizer.tokenize(t)
tokenizer.convert_tokens_to_ids(r), t, r

# Sometimes words exist of 2 tokens,
# We will presume it got the answer correct if it can guess the first one

([29506, 12370], 'skateboarding', ['skate', 'boarding</w>'])

In [105]:
len(data.pred.unique())

126

# Model

### Preprocessing

In [182]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
data = pd.read_pickle('Post_Processing_with_Encoding_ans_pred.pkl')
ohe_len = len(data.pred.unique())

enc = OneHotEncoder().fit(data.pred.to_numpy().reshape(-1,1))


## Preprocessing
data_train_test = data[(data.label == 5)]
data_eval_ea = data[(data.label == 4) | (data.label == 5)]
data_eval_ea.label[data_eval_ea.label == 4] = 1
data_eval_ea.label[data_eval_ea.label == 5] = 0

#data.ans_enc.apply(lambda x: x[0])

# Use pd dummies instead of GPT token id's since it is less hassle.


#data.label= np.random.choice([0,1],len(data.label))

data_train_test


c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-

ans                                                exp  \
363036001            cake  there is a pastry base which has been adorned ...   
75560012              yes                        there are clouds in the sky   
93236000               no                the train is sitting at the station   
126995004          tennis                 the man is holding a tennis racket   
243600002           chair                it is a flat and with a narrow seat   
302200002              no                                it is a city street   
4296004           kitchen                there is a stove and a refrigerator   
260150015        pancakes         it is cut in a slices and full of toppings   
118432009              no            the sky is clear and the people are dry   
519316004         bedroom                               there is a bed in it   
449872015             yes                          the sky is clear and blue   
272673007        bathroom                       there is a sink and a mirror   
279437000         bedroom                               there is a bed in it   
318284000          tennis                 the man is holding a tennis racket   
294537003             yes                                      there is snow   
3590002               yes                                      it is raining   
229234002              no                          the sky is clear and blue   
490878000          skiing                                      he is on skis   
398138004              no                          the sky is clear and blue   
166731002              no             the sky is clear and the ground is dry   
68114002              yes                                   the light is red   
428305002        bathroom                       there is a sink and a mirror   
482719002             yes                                         it is ripe   
547617003          tennis                      he is holding a tennis racket   
292227002   skateboarding                              he is on a skateboard   
280678007            fork                            it has tines at the end   
8981001           surfing                               he is on a surfboard   
463703001           pizza                         it is round and has cheese   
283190012         kitchen  there is a stove and refrigarator and a sink i...   
464178002         giraffe                       it has a long neck and spots   
...                   ...                                                ...   
409178007          skiing  the person is wearing skis and going down a hi...   
166868000              no                   there are many clouds in the sky   
340934001        bathroom                       there is a sink and a mirror   
110601012        bathroom                       there is a sink and a toilet   
250076002           sunny                          the sky is clear and blue   
129886000          winter                        there is snow on the ground   
296404007              no                          the sky is clear and blue   
61897000              yes  the sky is dark and the street lights are turn...   
422516001           pizza                   it is round and has cheese on it   
242513000              no                              it is an older design   
429598006             gas                                 it has gas burners   
526044021             cow                    it has horns and a cowlike face   
305800002             yes  there are people socializing and enjoying them...   
528136020             van          it is a large vehicle with a split on top   
318138001              no          she is smiling and playing a nintendo wii   
178911004           train                            it is on a track system   
482367001          tennis                 the man is holding a tennis racket   
365194001          tennis                      he is holding a tennis racket   
425077002             

In [183]:
# enc.transform(data.ans.to_numpy().reshape(-1,1))
# pd.Series(data.ans.unique()).diff(pd.Series(data.pred.unique()))


In [184]:
def pad_exp(exp):
    
    
    s = exp.shape
    out = np.pad(exp, ((0,0),(0,22-s[1]),(0,0)),'constant', constant_values = 0)
    
    return out

#fake = np.random.rand(918,22,768)

data_train_test.exp_enc = data_train_test.exp_enc.apply(pad_exp)
data_train_test.qstr_enc = data_train_test.qstr_enc.apply(pad_exp)


# ones, zeros = data.label.value_counts()

# ones/(ones+zeros)

# data.label.value_counts()

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Model building

In [185]:
from tensorflow import keras
from tensorflow.keras import layers


input_qstr = keras.Input(shape=(22,768))
flatten_qstr = layers.Flatten()(input_qstr)

input_exp = keras.Input(shape=(22,768))
flatten_exp = layers.Flatten()(input_exp)

concat = layers.concatenate(([flatten_qstr, flatten_exp]))

d = layers.Dense(64,activation = 'relu')(concat)
d = layers.Dense(64, activation = 'relu')(d)

output = layers.Dense(ohe_len, activation = 'softmax')(d)

model = keras.Model([input_qstr,input_exp],output)
model.summary()
model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 16896)        0           input_5[0][0]                    
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 16896)        0           input_6[0][0]                    
____________________________________________________________________________________________

### Training the answer guesser

In [186]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_train_test, random_state=42)


model.fit([np.vstack(train['qstr_enc'].to_numpy()),
          np.vstack(train['exp_enc'].to_numpy())],
          enc.transform(train.pred.to_numpy().reshape(-1,1)).todense(),
          batch_size=32,
          epochs=20)

Epoch 1/20
20/20 [==============================] - 0s 7ms/step - loss: 3.2925 - accuracy: 0.3667
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 1.3712 - accuracy: 0.6931
Epoch 3/20
20/20 [==============================] - 0s 5ms/step - loss: 0.7389 - accuracy: 0.8191
Epoch 4/20
20/20 [==============================] - 0s 5ms/step - loss: 0.3890 - accuracy: 0.9128
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.2074 - accuracy: 0.9612
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1244 - accuracy: 0.9725
Epoch 7/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0809 - accuracy: 0.9822
Epoch 8/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0668 - accuracy: 0.9871
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0624 - accuracy: 0.9774
Epoch 10/20
20/20 [==============================] - 0s 5ms/step - loss: 0.0621 - accuracy: 0.9806
Epoch 11/20
20/20 [

In [200]:
#evaluating:
from sklearn.metrics import f1_score
print(model.evaluate([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())],
          enc.transform(test.pred.to_numpy().reshape(-1,1)).todense()),  model.metrics_names)

predict = model.predict([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())])

np.hstack([enc.inverse_transform(predict), test.pred.to_numpy().reshape(-1,1)])

f1_score(enc.inverse_transform(predict), test.pred.to_numpy().reshape(-1,1),average = 'micro')


7/7 [==============================] - 0s 4ms/step - loss: 2.3604 - accuracy: 0.7440
[2.3603978157043457, 0.7439613342285156] ['loss', 'accuracy']


0.7439613526570048

## Part 2: predicting the label using pred and pred_pred haha

We have a model that predicts pred from qstr & exp. We will save this prediction in a row called pred_pred, and then compare whether a mismatch (pred_pred!=pred) occurs more often with label 0 or label 1.



In [334]:
data_eval_ea = data[(data.label == 4) | (data.label == 5)]
data_eval_ea.label[data_eval_ea.label == 4] = 1
data_eval_ea.label[data_eval_ea.label == 5] = 0

train_ea, test_ea = train_test_split(data_eval_ea, random_state=42)

# WERKT NIET
# def predict_row(row,model,encoder):
#     exp = np.vstack(row.exp_enc.apply(pad_exp).to_numpy())
#     qstr = np.vstack(row.qstr_enc.apply(pad_exp).to_numpy())
    
#     pred_enc = model.predict([exp,qstr])
#     pred = enc.inverse_transform(pred_enc)
#     return pred

# samp = data_eval_ea.sample()
# predict_row(samp,model,enc) == samp.pred.to_numpy()



# data_eval_ea['pred_pred'] = predict_row(data_eval_ea,model,enc)
#data_eval_ea['pred_ae'] = data_eval_ea['pred_ae']
lab0 = data_eval_ea[data_eval_ea.label==0]
lab1 = data_eval_ea[data_eval_ea.label==1]

from sklearn.metrics import accuracy_score, classification_report
'no_ae:',accuracy_score(lab0.pred.to_numpy().reshape(-1,1), lab0.pred_pred.to_numpy().reshape(-1,1)) , \
'ae:', accuracy_score(lab1.pred.to_numpy().reshape(-1,1), lab1.pred_pred.to_numpy().reshape(-1,1))

#classification_report(data_eval_ea.pred, data_eval_ea.pred_pred)
#Note: it just guesses yes most of the time

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ex

AttributeError: 'DataFrame' object has no attribute 'pred_pred'

In [260]:
data_eval_ea.exp_enc = data_eval_ea.exp_enc.apply(pad_exp)
data_eval_ea.qstr_enc = data_eval_ea.qstr_enc.apply(pad_exp)

predict = model.predict([np.vstack(data_eval_ea['qstr_enc'].to_numpy()),
          np.vstack(data_eval_ea['exp_enc'].to_numpy())])
                         

In [264]:
data_eval_ea['pred_pred'] = enc.inverse_transform(predict)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [267]:
pred = data_eval_ea[data_eval_ea.label==1].pred
pred_pred = data_eval_ea[data_eval_ea.label==1].pred_pred

In [288]:
print(accuracy_score(pred,pred_pred)) #36% accuracy 
data_eval_ea[(data_eval_ea.label == 1)& (data_eval_ea.pred==data_eval_ea.pred_pred)]
# Allmost all of the cases where the model could predict the true answer was yes/no questions when the 
# a-e mismatch label was assigned to the explanation

0.34782608695652173


ans                                                exp    pred  \
366421000      no                      the window is angled outwards      no   
372070010     yes  it is stopped at the station to let people on ...     yes   
432378003      no  it is in black and white and the old style is old     yes   
440937002     yes                    it has a touch screen and icons      no   
168231003     yes                                      there is snow     yes   
148531000      no         it is a stealth plane with an older design     yes   
332916003     yes             the sign on the sign is not in english     yes   
49115002   eating             it is reaching its head into the water  eating   
268641000     yes                           it is very old and faded      no   
10995004      yes                      the window is angled outwards      no   
486087000      no                      the window is angled outwards      no   
491481016     yes    the sky is gray and there are no clouds visible     yes   
158950005     yes  it is stopped at the station to let people on ...     yes   
35778002       no            the giraffes are in a unnatural exhibit      no   
534038010      no      the waves are pounding off the mountain rocks      no   
455735000      no                    the people are dressed formally      no   
135319000     yes                                       it is a city      no   
550055002      no                                it is a fighter jet      no   
542676002     yes                it is short and not to her sholders     yes   
492683000     yes                the window is rolled up all the way      no   
505814001      no                      his hair is touching his back      no   
448759002      no                          the sky is blue and clear      no   
242345007      no                      the window is angled outwards      no   
503412002     yes                      the window is sealed outwards      no   
108953005     yes                        it is surrounded by fencing     yes   
535506003      no                        she has a smile on her face      no   
215376006     yes  the man is wearing a suit and the woman a nice...     yes   
333677004      no               the comforter is flat and unwrinkled      no   
106356004     yes                the window is rolled up all the way      no   
488664001     yes  it is stopped at the station to let people on ...     yes   
528866001     yes                    there are no vegetables present     yes   
188386007     yes          the man is wearing a modern current shirt     yes   

                                               qstr  label  \
366421000                        is the window open      1   
372070010                       is the train moving      1   
432378003                  is this a modern picture      1   
440937002                      is this a smartphone      1   
168231003                 does he like the outdoors      1   
148531000                    is this a modern plane      1   
332916003                        is this in america      1   
49115002                   what is the animal doing      1   
268641000  does the red fire hydrant need new paint      1   
10995004                         is the window open      1   
486087000                        is the window open      1   
491481016                              is it cloudy      1   
158950005                       is the train moving      1   
35778002                              is this a zoo      1   
534038010                       is the water choppy      1   
455735000                 is this a black tie event      1   
135319000                is this a residential area      1   
550055002                  is this a military plane      1   
542676002             does the woman have long hair      1   
492683000                        is the window open      1   
505814001   does this man have any hair on his head      1   
448759002

In [272]:
data_eval_ea[(data_eval_ea.label == 1)& (data_eval_ea.pred!=data_eval_ea.pred_pred)]

ans                                                exp  \
563040007            no                      it is parked in a parking lot   
518438001          stop                                 the light is green   
579697002         china             the writing on the signs is in english   
178748001      electric                the burners are clearly gas burners   
385770001           yes          the room is dark and there is no sunlight   
441218002        skiing                   he is on a snowboard on the snow   
487607003            no  it is in color and the person is wearing moder...   
404886003        eating                     they are tired and laying down   
92804002         skiing                 they are on a snowy hill with skis   
243158005            no                  there are few personal belongings   
217522000            no  the woman is holding an umbrella and there are...   
207458018            no                             he is wearing a helmet   
49005014            yes       the players are wearing professional jersies   
314965000           yes              it is made of metal and is reflecting   
234684003        french                       the signs are not in english   
190291002         pizza  the food is circular and covered in chocolate ...   
111062001           yes                      the floor is covered in trash   
321886001            no                           the signs are in english   
518383006  snowboarding  they are on a snowy mountain with skis on thei...   
234785000           dog                   it is a large animal with a mane   
121414007            no                                        it is a cow   
308764000           owl                                it has a round beak   
301402000         chair                                it is a single seat   
216006003           yes                           the signs are in english   
446232001           yes                               he is on a snowboard   
354753002          stop                                 the light is green   
128014004            no   there are no personal items anywhere in the room   
46859005             no           the players are wearing official jersies   
296426000            no                                   the sign says it   
423508005            no         the sky is bright and there is no sunlight   
476398015          left  the fingers are lined up in a way a left hand ...   
531875001        carrot            it has a yellow skin and a green inside   
368222007         phone                            he is holding a wiimote   
177838001         right  the fingers are lined up in a way a left hand ...   
419624000            no  the train is followed by many cars for hauling...   
435037002        coffee         it is a dark colored color in a wine glass   
519801000           yes  the train is followed by many cars for hauling...   
157417002        desert  there is a sandy expanse that leads to the wat...   
132510005  snowboarding                 they are on a snowy hill with skis   
536370001  snowboarding             they are on a snowy mountain with skis   
294247002           yes      it is in the sky and appears to be taking off   
264572002            no           the players are wearing official jersies   
289870005           yes  the train is followed by many cars for hauling...   
578852005        french                       the signs are not in english   
17655000          couch                                        it is a bed   
579760003          beer                      the liquid is in a wine glass   
404071000           yes                 the train is letting passengers on   
13991003            gas                the burners are clearly gas burners   
167271003           yes    she is wearing a pair of glasses in the picture   
166277003            no                              it is a striped kitty   
264599001            no                   

In [287]:

# When the answer is guessed correctly, how often is it a yes/no question? I suspect this is higher for the a-e category
data_eval_ea[(data_eval_ea.pred==data_eval_ea.pred_pred) &\
             (data_eval_ea.pred_pred.isin(['yes','no']))]\
             .groupby('label').size() / data_eval_ea[(data_eval_ea.pred==data_eval_ea.pred_pred)].groupby('label').size() 

label
0    0.28886
1    0.96875
dtype: float64

In [291]:
#Do the check for test to see if it is also true for values the model hasn't seen yet
test_predict = model.predict([np.vstack(test['qstr_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())])
test['pred_pred'] = enc.inverse_transform(test_predict)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [294]:

# When the answer is guessed correctly, how often is it a yes/no question? I suspect this is higher for the a-e category

test[(test.pred==test.pred_pred) &\
             (test.pred_pred.isin(['yes','no']))]\
             .groupby('label').size() / test[(test.pred==test.pred_pred)].groupby('label').size() 

label
5    0.25974
dtype: float64

In [347]:
# Same for lab1
lab1.exp_enc = lab1.exp_enc.apply(pad_exp)
lab1.qstr_enc = lab1.qstr_enc.apply(pad_exp)

lab1_predict = model.predict([np.vstack(lab1['qstr_enc'].to_numpy()),
          np.vstack(lab1['exp_enc'].to_numpy())])
lab1['pred_pred'] = enc.inverse_transform(lab1_predict)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [348]:
test['label'] = 0

finaltest = pd.concat([lab1,test])
finaltest['pred_label'] = (finaltest.pred_pred != finaltest.pred) #model couldnt predict true ans --> error detected
finaltest['label'] = finaltest['label'] == 1 #True == A-E error detected by researcher
pred_label = finaltest['pred_label'].to_numpy().reshape(-1,1)
true_label = finaltest['label'].to_numpy().reshape(-1,1)

print(classification_report(pred_label,true_label))
np.hstack((pred_label,true_label))


              precision    recall  f1-score   support

       False       0.74      0.83      0.78       186
        True       0.65      0.53      0.59       113

    accuracy                           0.72       299
   macro avg       0.70      0.68      0.68       299
weighted avg       0.71      0.72      0.71       299



c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


array([[False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

In [353]:

len(lab1), len(test)

(92, 207)